This code utlizes a pdf released by Eunomia and Ball Corporation in March 2021, which describes packaging waste generation, recycling, and disposal per capita per U.S. State. The data can be found at this link: https://www.ball.com/getattachment/37f5f87f-d462-44c5-913f-d3075754741a/50-States-of-Recycling-Eunomia-Report-Final-Published-March-30-2021-UPDATED-v2.pdf


Importantly, this extracts all data corresponding to HDPE Bottles, PP, Rigids #3-7 and examines how much plastic is **generated**. ***If users would like to extract data regarding recycling rates per county, refer to a separate python script provided in the github. ***

To use this code users must download the pdf and import the data into google colab or another python IDE.

**Required Datasets**
1. **'50statesrecycling.pdf'** downloaded from https://www.ball.com/getattachment/37f5f87f-d462-44c5-913f-d3075754741a/50-States-of-Recycling-Eunomia-Report-Final-Published-March-30-2021-UPDATED-v2.pdf
2. **'fips2county.tsv'** This is a database of all U.S. County FIPS codes. Downloaded from:  https://github.com/ChuckConnell/articles/blob/master/fips2county.tsv
(if users use an FIPS dataset of their choosing, users must be aware that when merging U.S. counties with the provided FIPS tsv file, users must be cautious that certain counties with special characters such as Doña Ana, New Mexico may not automatically be matched due to special characters in the name. Since Doña Ana was the only county in which this occured, the github code provided with this report accounts for only this county. If a different FIPS dataset is used to merge the U.S. Census county-level data, users should be aware of this potential issue.) Furthermore, some datasets include spaces in county names while others do not. This will affect how the datasets merge and influence which counties show up on a map which uses this data.
3. **'population.xlsx'** United States Census Bureau population data is taken from: https://www.census.gov/data/tables/time-series/demo/popest/2020s-counties-total.html

4. **'CT County Population Data.xlsx'** The United States Census does not report county-level population data for connecticut, so 2021 data is used from:  https://portal.ct.gov/SOTS/Register-Manual/Section-VII/Population-of-Connecticut-by-Counties


**Produced Datasets**
This code produces the following files
1. **'Plastic_Generation_County.csv'** This is a master datasheet depicting each county's plastic generation in 2021. It is created by using dataset 4 and the U.S. Population data.
2.**'GENERATED State total plastic.xlsx'** this is an excel file which has a sheet for each state's county-level plastic generation data. It is the 'Plastic_Generation_County.csv' organized by state
3. **'State lbs generated plastic per capita'** This is a csv file of each state's lbs per capita. It is used to create datasets 4. It is the raw data from the Ball Coproration pdf.
4. **'Tons and lbs generated plastic per capita.csv’** This dataset depicts the plastic generation per U.S. state in tons and lbs. It is the basis of datasets 1 and 2.


In [ ]:
import pandas as pd
!pip install tabula-py
import tabula

To scrape the data from the Ball Corporation pdf, code had to be developed for each of the three states on pages 206-22. As shown below:


#First Table Code

In [ ]:
#This is an example of extracting the 'HDPE Bottles', 'PP', 'Rigids #3-7' data for Alabama

file1 = '50statesrecycling.pdf'
table = tabula.read_pdf(file1,pages=206)
a = table[0]
name = 'Alabama'
new_columns = [f'Plastics in {name}', 'HDPE Bottles', 'PP', 'Rigids #3-7']
a_new = a[['LBS/CAPITA', 'Unnamed: 5', 'Unnamed: 6', 'Unnamed: 7']].copy()
a_new.columns = new_columns
filtered_df = a_new[(a_new[f'Plastics in {name}'] == 'GENERATED')].copy()
filtered_df['sum (lbs per capita)'] = filtered_df['HDPE Bottles'] + filtered_df['PP'] + filtered_df['Rigids #3-7']
sum = filtered_df.loc[0,'sum (lbs per capita)']
print(sum)
filtered_df.head()

#If users would like to extract all of the plastics (which means including the PET values), the following code can be used:

###PLASTICS TOTAL
# file1 = '50statesrecycling.pdf'
# table = tabula.read_pdf(file1,pages=206)
# a = table[0]
# # a
# name = 'Alabama'
# new_columns = [f'Plastics in {name}', 'Plastics Total']
# a_new = a[['LBS/CAPITA', 'Unnamed: 2']].copy()
# a_new.columns = new_columns
# Generated_df = a_new[(a_new[f'Plastics in {name}'] == 'GENERATED')].copy()
# # filtered_df['sum (lbs per capita)'] = filtered_df['HDPE Bottles'] + filtered_df['PP'] + filtered_df['Rigids #3-7']
# sum = Generated_df.iloc[0,1]
# print(sum)
# Generated_df.head()


23.6


,Plastics in Alabama,HDPE Bottles,PP,Rigids #3-7,sum (lbs per capita)
0,GENERATED,12.5,7.0,4.1,23.6


In [ ]:
file1 = '50statesrecycling.pdf'
def firsttable(name, pagenumber, file1):
  '''This function extracts the 'HDPE Bottles', 'PP', 'Rigids #3-7' lbs/capita generated for states located first on pages 206-222 in the Ball Corporation pdf.
  Inputs:
  name: the state name
  pagenumber: page number on which the state's data is located
  file1: the desired pdf

  Outputs:
  filtered_df: data frame of the of 'HDPE Bottles', 'PP', 'Rigids #3-7' lbs/capita generated for name and their sum
  sum: sum of 'HDPE Bottles', 'PP', 'Rigids #3-7' lbs/capita generated for name
  '''

  #read in all tables on this page, take first table and extract desired columns
  tables = tabula.read_pdf(file1,pages=pagenumber)
  a = tables[0]
  a_new = a[['LBS/CAPITA', 'Unnamed: 5', 'Unnamed: 6', 'Unnamed: 7']].copy()

  #rename columns
  new_columns = [f'Plastics in {name}', 'HDPE Bottles', 'PP', 'Rigids #3-7']
  a_new.columns = new_columns

  #grab only generated
  filtered_df = a_new[(a_new[f'Plastics in {name}'] == 'GENERATED')].copy()
  filtered_df['sum (lbs per capita)'] = filtered_df['HDPE Bottles'] + filtered_df['PP'] + filtered_df['Rigids #3-7']
  sum = filtered_df.loc[0,'sum (lbs per capita)']
  return filtered_df, sum


#If users would like to extract all of the plastics (which means including the PET values), the following function can be used instead:
###PLASTICS TOTAL
# def firsttable(name, pagenumber, file1):
 '''This function extracts the all plastics lbs/capita generated for states located first on pages 206-222 in the Ball Corporation pdf.
  Inputs:
  name: the state name
  pagenumber: page number on which the state's data is located
  file1: the desired pdf

  Outputs:
  Generated_df: data frame of the of all plastics lbs/capita generated for name and their sum
  sum: sum of all plastics lbs/capita generated for name
  '''
#   tables = tabula.read_pdf(file1,pages=pagenumber)
#   a = tables[0]
#   new_columns = [f'Plastics in {name}', 'Plastics Total']
#   a_new = a[['LBS/CAPITA', 'Unnamed: 2']].copy()
#   a_new.columns = new_columns
#   Generated_df = a_new[(a_new[f'Plastics in {name}'] == 'GENERATED')].copy()
#   sum = Generated_df.iloc[0,1]
#   return Generated_df, sum


#Dictionary to store sums

In [ ]:
#Dictionary to store all generated values.
allstates = {}

In [ ]:
Alabamadf, Alabamasum  = firsttable('Alabama', 206, file1)
#display(Alabamadf)
allstates['Alabama'] = Alabamasum

Arkansasdf, Arkansassum  = firsttable('Arkansas', 207, file1)
#display(Arkansasdf)
allstates['Arkansas'] = Arkansassum

Connecticutdf, Connecticutsum  = firsttable('Connecticut', 208, file1)
#display(Connecticutdf)
allstates['Connecticut'] = Connecticutsum

Georgiadf, Georgiasum  = firsttable('Georgia', 209, file1)
#display(Georgiadf)
allstates['Georgia'] = Georgiasum

Illinoisdf, Illinoissum  = firsttable('Illinois', 210, file1)
#display(Illinoisdf)
allstates['Illinois'] = Illinoissum

Kansasdf, Kansassum  = firsttable('Kansas', 211, file1)
#display(Kansasdf)
allstates['Kansas'] = Kansassum

Mainedf, Mainesum  = firsttable('Maine', 212, file1)
#display(Mainedf)
allstates['Maine'] = Mainesum

Michigandf, Michigansum  = firsttable('Michigan', 213, file1)
#display(Michigandf)
allstates['Michigan'] = Michigansum

Missouridf, Missourisum  = firsttable('Missouri', 214, file1)
#display(Missouridf)
allstates['Missouri'] = Missourisum

Nevadadf, Nevadasum  = firsttable('Nevada', 215, file1)
#display(Nevadadf)
allstates['Nevada'] = Nevadasum

New_Mexicodf, New_Mexicosum  = firsttable('New Mexico', 216, file1)
#display(New_Mexicodf)
allstates['New Mexico'] = New_Mexicosum

North_Dakotadf, North_Dakotasum  = firsttable('North Dakota', 217, file1)
#display(North_Dakotadf)
allstates['North Dakota'] = North_Dakotasum

Oregondf, Oregonsum  = firsttable('Oregon', 218, file1)
#display(Oregondf)
allstates['Oregon'] = Oregonsum

South_Carolinadf, South_Carolinasum  = firsttable('South Carolina', 219, file1)
#display(South_Carolinadf)
allstates['South Carolina'] = South_Carolinasum

Texasdf, Texassum  = firsttable('Texas', 220, file1)
#display(Texasdf)
allstates['Texas'] = Texassum

Virginiadf, Virginiasum  = firsttable('Virginia', 221, file1)
#display(Virginiadf)
allstates['Virginia'] = Virginiasum

Wisconsindf, Wisconsinsum  = firsttable('Wisconsin', 222, file1)
#display(Wisconsindf)
allstates['Wisconsin'] = Wisconsinsum

<class 'pandas.core.series.Series'>
<class 'pandas.core.series.Series'>
<class 'pandas.core.series.Series'>
<class 'pandas.core.series.Series'>
<class 'pandas.core.series.Series'>
<class 'pandas.core.series.Series'>
<class 'pandas.core.series.Series'>
<class 'pandas.core.series.Series'>
<class 'pandas.core.series.Series'>
<class 'pandas.core.series.Series'>
<class 'pandas.core.series.Series'>
<class 'pandas.core.series.Series'>
<class 'pandas.core.series.Series'>
<class 'pandas.core.series.Series'>
<class 'pandas.core.series.Series'>
<class 'pandas.core.series.Series'>
<class 'pandas.core.series.Series'>


#Second Table Code

In [ ]:
#example code displaying data generation for Alaska
file1 = '50statesrecycling.pdf'
table = tabula.read_pdf(file1,pages=206)
a = table[1]
name = 'Alaska'
a
filtered_df = a[a[f'Unnamed: 0'] == 'GENERATED'].iloc[[1]].copy()
filtered_df.rename(columns={'Unnamed: 0': f'Plastics in {name}'}, inplace = True )
a_new = filtered_df[[f'Plastics in {name}', 'HDPE Bottles', 'PP', 'Rigids #3-7']].copy()
a_new['sum (lbs per capita)'] = pd.to_numeric(a_new['HDPE Bottles']) + pd.to_numeric(a_new['PP']) + pd.to_numeric(a_new['Rigids #3-7'])
a_new.head()

,Plastics in Alaska,HDPE Bottles,PP,Rigids #3-7,sum (lbs per capita)
11,GENERATED,11.9,7.0,5.8,24.7


In [ ]:

###only certan plastic types
file1 = '50statesrecycling.pdf'
def secondtable(name, pagenumber, file1):
  #read all tables on pagenumber and extract second one
  table = tabula.read_pdf(file1,pages=pagenumber)
  a = table[1]

  #extract row of generated values
  filtered_df = a[a[f'Unnamed: 0'] == 'GENERATED'].iloc[[1]].copy()
  filtered_df.rename(columns={'Unnamed: 0': f'Plastics in {name}'}, inplace = True )

  a_new = filtered_df[[f'Plastics in {name}', 'HDPE Bottles', 'PP', 'Rigids #3-7']].copy()
  a_new['sum (lbs per capita)'] = pd.to_numeric(a_new['HDPE Bottles']) + pd.to_numeric(a_new['PP']) + pd.to_numeric(a_new['Rigids #3-7'])
  a_new = a_new.reset_index()
  sum = a_new.loc[0,'sum (lbs per capita)']
  a_new.head()
  return a_new, sum


###all plastic types
# def secondtable(name, pagenumber, file1):
#   file1 = '50statesrecycling.pdf'
#   table = tabula.read_pdf(file1,pages=pagenumber)
#   a = table[1]
#   filtered_df = a[a[f'Unnamed: 0'] == 'GENERATED'].iloc[[1]].copy()
#   filtered_df.rename(columns={'Unnamed: 0': f'Plastics in {name}'}, inplace = True )
#   a_new = filtered_df[[f'Plastics in {name}', 'Plastics Total']].copy()
#   sum = a_new.iloc[0,1]
#   return a_new, sum


In [ ]:
Alaskadf, Alaskasum  = secondtable('Alaska', 206, file1)
#display(Alaskadf)
allstates['Alaska'] = Alaskasum

Californiadf, Californiasum = secondtable('California', 207, file1)
#display(Californiadf)
allstates['California'] = Californiasum

Delawaredf, Delawaresum  = secondtable('Delaware', 208, file1)
#display(Delawaredf)
allstates['Delaware'] = Delawaresum

Hawaiidf, Hawaiisum  = secondtable('Hawaii', 209, file1)
#display(Hawaiidf)
allstates['Hawaii'] = Hawaiisum

Indianadf, Indianasum  = secondtable('Indiana', 210, file1)
#display(Indianadf)
allstates['Indiana'] = Indianasum

Kentuckydf, Kentuckysum  = secondtable('Kentucky', 211, file1)
#display(Kentuckydf)
allstates['Kentucky'] = Kentuckysum

Marylanddf, Marylandsum  = secondtable('Maryland', 212, file1)
#display(Marylanddf)
allstates['Maryland'] = Marylandsum

Minnesotadf, Minnesotasum  = secondtable('Minnesota', 213, file1)
#display(Minnesotadf)
allstates['Minnesota'] = Minnesotasum

Montanadf, Montanasum  = secondtable('Montana', 214, file1)
#display(Montanadf)
allstates['Montana'] = Montanasum

New_Hampshiredf, New_Hampshiresum  = secondtable('New Hampshire', 215, file1)
#display(New_Hampshiredf)
allstates['New Hampshire'] = New_Hampshiresum

New_Yorkdf, New_Yorksum  = secondtable('New York', 216, file1)
#display(New_Yorkdf)
allstates['New York'] = New_Yorksum

Ohiodf, Ohiosum  = secondtable('Ohio', 217, file1)
#display(Ohiodf)
allstates['Ohio'] = Ohiosum

Pennsylvaniadf, Pennsylvaniasum  = secondtable('Pennsylvania', 218, file1)
#display(Pennsylvaniadf)
allstates['Pennsylvania'] = Pennsylvaniasum

South_Dakotadf, South_Dakotasum  = secondtable('South Dakota', 219, file1)
#display(South_Dakotadf)
allstates['South Dakota'] = South_Dakotasum

Utahdf, Utahsum  = secondtable('Utah', 220, file1)
#display(Utahdf)
allstates['Utah'] = Utahsum

Washingtondf, Washingtonsum  = secondtable('Washington', 221, file1)
#display(Washingtondf)
allstates['Washington'] = Washingtonsum

Wyomingdf, Wyomingsum  = secondtable('Wyoming', 222, file1)
#display(Wyomingdf)
allstates['Wyoming'] = Wyomingsum


#Extracting Data from Third Table

In [ ]:
#extracting data from the third table example:

file1 = '50statesrecycling.pdf'
table = tabula.read_pdf(file1,pages=206)
a = table[3]
name = 'Arizona'
filtered_df = a[(a[f'Unnamed: 0'] == 'GENERATED')].copy()
filtered_df.rename(columns={'Unnamed: 0': f'Plastics in {name}'}, inplace = True )
a_new = filtered_df[[f'Plastics in {name}', 'HDPE Bottles', 'PP', 'Rigids #3-7']].copy()
filtered_df['sum (lbs per capita)'] = filtered_df['HDPE Bottles'] + filtered_df['PP'] + filtered_df['Rigids #3-7']

a_new.head()


file1 = '50statesrecycling.pdf'
table = tabula.read_pdf(file1,pages=206)
a = table[3]
name = 'Arizona'
filtered_df = a[a[f'Unnamed: 0'] == 'GENERATED'].iloc[[0]].copy()
filtered_df.rename(columns={'Unnamed: 0': f'Plastics in {name}'}, inplace = True )
a_new = filtered_df[[f'Plastics in {name}', 'Plastics Total']].copy()
sum = a_new.iloc[0,1]

,Plastics in Arizona,HDPE Bottles,PP,Rigids #3-7
1,GENERATED,11.9,7.0,4.1


Functions

In [ ]:
###Plastic Type by Category
file1 = '50statesrecycling.pdf'
def thirdtable(name, pagenumber, file1):

  #read in the pagenumber and extract the FOURTH table
  table = tabula.read_pdf(file1,pages=pagenumber)
  a = table[3]

  #adjust column names as needed
  filtered_df = a[(a[f'Unnamed: 0'] == 'GENERATED')].copy()
  filtered_df.rename(columns={'Unnamed: 0': f'Plastics in {name}'}, inplace = True )
  a_new = filtered_df[[f'Plastics in {name}', 'HDPE Bottles', 'PP', 'Rigids #3-7']].copy()
  a_new['sum (lbs per capita)'] = a_new['HDPE Bottles'] + a_new['PP'] + a_new['Rigids #3-7']
  a_new = a_new.reset_index()
  sum = a_new.loc[0,'sum (lbs per capita)']
  a_new.head()
  return a_new, sum


# ##All Plastics
# def thirdtable(name, pagenumber, file1):
#   file1 = '50statesrecycling.pdf'
#   table = tabula.read_pdf(file1,pages=pagenumber)
#   a = table[3]
#   filtered_df = a[a[f'Unnamed: 0'] == 'GENERATED'].iloc[[0]].copy()
#   filtered_df.rename(columns={'Unnamed: 0': f'Plastics in {name}'}, inplace = True )
#   a_new = filtered_df[[f'Plastics in {name}', 'Plastics Total']].copy()
#   sum = a_new.iloc[0,1]
#   return a_new, sum


In [ ]:
Arizonadf, Arizonadfsum = thirdtable('Arizona', 206, file1)
#display(Arizonadf)
allstates['Arizona'] = Arizonadfsum

Coloradodf, Coloradosum = thirdtable('Colorado', 207, file1)
#display(Coloradodf)
allstates['Colorado'] = Coloradosum

Floridadf, Floridasum  = thirdtable('Florida', 208, file1)
#display(Floridadf)
allstates['Florida'] = Floridasum

Idahodf, Idahosum  = thirdtable('Idaho', 209, file1)
#display(Idahodf)
allstates['Idaho'] = Idahosum

Iowadf, Iowasum  = thirdtable('Iowa', 210, file1)
#display(Iowadf)
allstates['Iowa'] = Iowasum

Louisianadf, Louisianasum  = thirdtable('Louisiana', 211, file1)
#display(Louisianadf)
allstates['Louisiana'] = Louisianasum

Massachusettsdf, Massachusettssum  = thirdtable('Massachusetts', 212, file1)
#display(Massachusettsdf)
allstates['Massachusetts'] = Massachusettssum

Mississippidf, Mississippisum  = thirdtable('Mississippi', 213, file1)
#display(Mississippidf)
allstates['Mississippi'] = Mississippisum

Nebraskadf, Nebraskasum  = thirdtable('Nebraska', 214, file1)
#display(Nebraskadf)
allstates['Nebraska'] = Nebraskasum

New_Jerseydf, New_Jerseysum  = thirdtable('New Jersey', 215, file1)
#display(New_Jerseydf)
allstates['New Jersey'] = New_Jerseysum

North_Carolinadf, North_Carolinasum  = thirdtable('North Carolina', 216, file1)
#display(North_Carolinadf)
allstates['North Carolina'] = North_Carolinasum

Oklahomadf, Oklahomasum  = thirdtable('Oklahoma', 217, file1)
#display(Oklahomadf)
allstates['Oklahoma'] = Oklahomasum

Rhode_Islanddf, Rhode_Islandsum  = thirdtable('Rhode Island', 218, file1)
#display(Rhode_Islanddf)
allstates['Rhode Island'] = Rhode_Islandsum

Tennesseedf, Tennesseesum  = thirdtable('Tennessee', 219, file1)
#display(Tennesseedf)
allstates['Tennessee'] = Tennesseesum

Vermontdf, Vermontsum  = thirdtable('Vermont', 220, file1)
#display(Vermontdf)
allstates['Vermont'] = Vermontsum

West_Virginiadf, West_Virginiasum  = thirdtable('West Virginia', 221, file1)
#display(West_Virginiadf)
allstates['West Virginia'] = West_Virginiasum

In [ ]:
#checking that all states have data associated with them.
states = [
    'Alabama', 'Alaska', 'Arizona', 'Arkansas', 'California', 'Colorado', 'Connecticut', 'Delaware',
    'Florida', 'Georgia', 'Hawaii', 'Idaho', 'Illinois', 'Indiana', 'Iowa', 'Kansas', 'Kentucky',
    'Louisiana', 'Maine', 'Maryland', 'Massachusetts', 'Michigan', 'Minnesota', 'Mississippi', 'Missouri',
    'Montana', 'Nebraska', 'Nevada', 'New Hampshire', 'New Jersey', 'New Mexico', 'New York',
    'North Carolina', 'North Dakota', 'Ohio', 'Oklahoma', 'Oregon', 'Pennsylvania', 'Rhode Island',
    'South Carolina', 'South Dakota', 'Tennessee', 'Texas', 'Utah', 'Vermont', 'Virginia', 'Washington',
    'West Virginia', 'Wisconsin', 'Wyoming'
]
for state in states:
  if state in allstates:
    pass
  else:
    print(state)



#Take all extracted values and export them to a csv

In [ ]:
statepercapita = pd.DataFrame()
statepercapita['State'] = list(allstates.keys())
statepercapita['sum (lbs per capita)'] = list(allstates.values())
statepercapita.to_csv('State lbs per capita', index=False)
statepercapita.head()

Once the code above has been run once, the following code can run without the previous code being run. For simplicity, it is kept in the same python code, so users can easily change the extracted data and the associated county-level data

In [ ]:
#reading in the population data
file_path = 'population.xlsx'
population = pd.read_excel(file_path, header = 3, usecols=[0,2,3,4])
population.rename(columns={'Unnamed: 0': f'County'}, inplace = True )
population.drop(0,inplace = True)
population.head()
population = population.iloc[:3144]
population[['County', 'State']] = population['County'].str.split(', ', expand=True)
population['County'] = population['County'].str.strip('.')
population.head()

,County,2020,2021,2022,State
1,Autauga County,58902.0,59210.0,59759.0,Alabama
2,Baldwin County,233219.0,239361.0,246435.0,Alabama
3,Barbour County,24960.0,24539.0,24706.0,Alabama
4,Bibb County,22183.0,22370.0,22005.0,Alabama
5,Blount County,59102.0,59085.0,59512.0,Alabama


In [ ]:
#read in the per capita values by state and convert lbs per capita to tons
statepercapita = pd.read_csv('State lbs generated plastic per capita.csv')
statepercapita['sum (tons per capita)'] = statepercapita['sum (lbs per capita)']*0.0005
statepercapita.to_csv('Tons and lbs generated plastic per capita.csv')
statepercapita.head()

,State,sum (lbs per capita),sum (tons per capita)
0,Alabama,23.6,0.01180
1,Arkansas,25.0,0.01250
2,Connecticut,21.3,0.01065
3,Georgia,27.6,0.01380
4,Illinois,25.4,0.01270


In [ ]:
states = [
    'Alabama', 'Alaska', 'Arizona', 'Arkansas', 'California', 'Colorado', 'Connecticut', 'Delaware',
    'Florida', 'Georgia', 'Hawaii', 'Idaho', 'Illinois', 'Indiana', 'Iowa', 'Kansas', 'Kentucky',
    'Louisiana', 'Maine', 'Maryland', 'Massachusetts', 'Michigan', 'Minnesota', 'Mississippi', 'Missouri',
    'Montana', 'Nebraska', 'Nevada', 'New Hampshire', 'New Jersey', 'New Mexico', 'New York',
    'North Carolina', 'North Dakota', 'Ohio', 'Oklahoma', 'Oregon', 'Pennsylvania', 'Rhode Island',
    'South Carolina', 'South Dakota', 'Tennessee', 'Texas', 'Utah', 'Vermont', 'Virginia', 'Washington',
    'West Virginia', 'Wisconsin', 'Wyoming'
]


In [ ]:
#Using 2022 data for each county and making a dataframe for each state.
statepopdfs = {}

#import the county codes
FIPS = pd.read_csv('fips2county.tsv', delimiter='\t', dtype={'CountyFIPS': str})
FIPS.rename(columns={'CountyName': f'County', 'StateName': 'State', 'CountyFIPS': 'FIPS'}, inplace = True )

#making sure Doña Ana County is written the same as the population data
FIPS.loc[(FIPS['County'] == 'DoÃ±a Ana') & (FIPS['State'] == 'New Mexico'), 'County'] = 'Doña Ana'

#merging each state's data with the FIPS codes and multiplying the county population by the tons generated per caita.
for state in states:
  #read in the state's population data
  popdf = population[population['State'] == f'{state}'].copy()
  statepopdfs[f'{state}'] = popdf
  sum = statepercapita[statepercapita['State'] == f'{state}'].iloc[0,1].copy()
  popdf['lbs per capita (state)'] = sum
  popdf['lbs generated in county (2022)'] = popdf['lbs per capita (state)'] * popdf[2022]
  popdf['County'] = popdf['County'].str.replace(' County', '')
  popdf['tons generated in county (2022)'] = popdf['lbs generated in county (2022)'] *.0005
  merged_popdf = pd.merge(popdf, FIPS, on=['County', 'State'])

  #the following code is used to identify if any merge between the two datasets failed.
  # It should identify that Connecticut's length after the merge is 0, which indicates that there is a difference in either FIPS codes or county names.
  #(in this case it's the county names)
  if len(merged_popdf) != len(popdf):
    print("before merge", len(popdf))
    print(state)
    print("State",state)
    print("length", len(merged_popdf))

  statepopdfs[f'{state}'] = merged_popdf

#exporting the dataframes into an excel file and accounting for the connecticut values
file_name = 'GENERATED State total plastic.xlsx'
with pd.ExcelWriter(file_name) as writer:
  for state in states:
    if state == 'Connecticut':
      df = pd.read_excel('CT County Population Data.xlsx', sheet_name='CT')
    else:
      df = statepopdfs[f'{state}']
    df.to_excel(writer, sheet_name=f'{state}', index=False)


#because working with excel sheets can be hard, one master csv of all counties is created below. The length should match the number of US counties.
xls = pd.ExcelFile('GENERATED State total plastic.xlsx')
Everything = {}

for sheet in xls.sheet_names:
    data = xls.parse(sheet)
    Everything[sheet] = data

GENERATED_ONECSV = pd.concat(Everything.values())

# Write the concatenated DataFrame to a CSV file
GENERATED_ONECSV.to_csv('Plastic_Generation_County.csv', index=False)

before merge 9
Connecticut
State Connecticut
length 0
